# 05 - Backtest Analysis

Run backtests and analyze results.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pathlib import Path

from backtest import BacktestEngine, BacktestReport, PerformanceMetrics, Signal
from backtest.walk_forward import WalkForwardOptimizer
from optimization import OptunaOptimizer

plt.style.use('dark_background')
sns.set_palette('husl')

In [ ]:
# Load configuration
config_path = Path('../config/default.yaml')
with open(config_path) as f:
    config = yaml.safe_load(f)

print("Backtest Configuration:")
bt_config = config['backtest']
print(f"  Initial capital: ${bt_config['initial_capital']:,}")
print(f"  Slippage type: {bt_config['slippage']['type']}")
print(f"  Taker fee: {bt_config['fees']['taker']*100:.2f}%")

In [ ]:
# Load or create data
SYMBOL = config['market']['symbol']
features_path = Path('../data/processed') / f'{SYMBOL}_features.parquet'

if features_path.exists():
    data = pd.read_parquet(features_path)
else:
    # Create synthetic data
    print("Creating synthetic data...")
    n = 10000
    np.random.seed(42)
    returns = np.random.randn(n) * 0.001
    prices = 50000 * np.exp(np.cumsum(returns))
    
    data = pd.DataFrame({
        'open': prices * (1 + np.random.randn(n) * 0.0001),
        'high': prices * (1 + np.abs(np.random.randn(n) * 0.0005)),
        'low': prices * (1 - np.abs(np.random.randn(n) * 0.0005)),
        'close': prices,
        'volume': np.random.randint(10, 100, n) * 0.1,
        'rsi': 50 + np.random.randn(n) * 15,
        'atr': prices * 0.02,
    }, index=pd.date_range(start='2024-01-01', periods=n, freq='1min'))

print(f"Data shape: {data.shape}")

## Define Strategies

In [ ]:
# RSI Mean Reversion Strategy
def rsi_strategy(features, context):
    rsi = features.get('rsi', 50)
    
    # Get thresholds from config
    overbought = config['features']['technical']['rsi']['overbought']
    oversold = config['features']['technical']['rsi']['oversold']
    
    if rsi < oversold:
        return Signal(direction=1, confidence=0.7, sl_mult=1.5, tp_mult=2.0)
    elif rsi > overbought:
        return Signal(direction=-1, confidence=0.7, sl_mult=1.5, tp_mult=2.0)
    return Signal(direction=0)

# Momentum Strategy
def momentum_strategy(features, context):
    rsi = features.get('rsi', 50)
    
    if rsi > 55:
        return Signal(direction=1, confidence=0.6, sl_mult=1.0, tp_mult=1.5)
    elif rsi < 45:
        return Signal(direction=-1, confidence=0.6, sl_mult=1.0, tp_mult=1.5)
    return Signal(direction=0)

# Hold Strategy (baseline)
def hold_strategy(features, context):
    return Signal(direction=0)

strategies = {
    'RSI Mean Reversion': rsi_strategy,
    'Momentum': momentum_strategy,
    'Hold (Baseline)': hold_strategy,
}

## Run Backtests

In [ ]:
# Create backtest engine
engine = BacktestEngine(config)

In [ ]:
# Run all strategies
results = engine.run_multiple(data, strategies, data)

# Compare results
comparison = engine.compare_results(results)
display(comparison)

## Analyze Results

In [ ]:
# Plot equity curves
fig, ax = plt.subplots(figsize=(14, 6))

for name, result in results.items():
    equity = result.get('equity_curve', pd.Series())
    if len(equity) > 0:
        ax.plot(equity.values, label=name, alpha=0.8)

ax.axhline(y=config['backtest']['initial_capital'], color='white', linestyle='--', alpha=0.5, label='Initial Capital')
ax.set_xlabel('Time Steps')
ax.set_ylabel('Equity ($)')
ax.set_title('Strategy Equity Curves')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Metrics comparison chart
metrics_to_plot = ['sharpe_ratio', 'sortino_ratio', 'max_drawdown', 'win_rate', 'profit_factor']

fig, axes = plt.subplots(1, len(metrics_to_plot), figsize=(16, 4))

for i, metric in enumerate(metrics_to_plot):
    values = [results[name].get(metric, 0) for name in strategies.keys()]
    axes[i].bar(range(len(strategies)), values)
    axes[i].set_xticks(range(len(strategies)))
    axes[i].set_xticklabels(list(strategies.keys()), rotation=45, ha='right')
    axes[i].set_title(metric.replace('_', ' ').title())

plt.tight_layout()
plt.show()

In [ ]:
# Detailed report for best strategy
best_strategy = comparison.loc[comparison['sharpe_ratio'].idxmax(), 'strategy']
best_results = results[best_strategy]

print(f"Best Strategy: {best_strategy}\n")

report = BacktestReport.from_backtest_results(best_results)
report.print_summary()

In [ ]:
# Trade analysis
trades = best_results.get('trades', pd.DataFrame())
if len(trades) > 0:
    print(f"\nTrade Analysis ({len(trades)} trades):")
    print(trades.describe())

## Drawdown Analysis

In [ ]:
# Calculate drawdown
equity = best_results.get('equity_curve', pd.Series())
if len(equity) > 0:
    peak = equity.expanding().max()
    drawdown = (equity - peak) / peak * 100
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Equity
    axes[0].plot(equity.values, color='cyan')
    axes[0].fill_between(range(len(equity)), equity.values, alpha=0.3)
    axes[0].set_title('Equity Curve')
    axes[0].set_ylabel('Equity ($)')
    
    # Drawdown
    axes[1].fill_between(range(len(drawdown)), drawdown.values, 0, color='red', alpha=0.5)
    axes[1].set_title('Drawdown')
    axes[1].set_ylabel('Drawdown (%)')
    axes[1].set_xlabel('Time Steps')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Maximum Drawdown: {drawdown.min():.2f}%")

## Save Reports

In [ ]:
# Save reports
report_dir = Path('../data/reports')
report_dir.mkdir(parents=True, exist_ok=True)

report.save_all(report_dir)
print(f"Reports saved to {report_dir}")

In [ ]:
# Save comparison
comparison.to_csv(report_dir / 'strategy_comparison.csv', index=False)
print(f"Strategy comparison saved")